In [23]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy.orm import Session


In [11]:
from sqlalchemy.ext.declarative import declarative_base
base = automap_base()
engine = create_engine(f"sqlite:///") 

In [24]:
base.metadata.create_all(engine) #combines engine and base

C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.salaries, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)


InvalidRequestError: Table 'salaries' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [22]:
session 
# salary_df = pd.DataFrame(salaries)

In [21]:
# 2. Create a histogram to visualize the most common salary ranges for employees.

# plt.plot()


# 3. Create a bar chart of average salary by title.